In [ ]:
from ..betting import helpers
import pybaseball as pyb, pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
pyb.cache.enable()
import numpy as np

: 

In [ ]:
# parse the database "pitch_arsenal2023.db" and print the table structure
import sqlite3
# connect to database
con = sqlite3.connect('../../data/databases/2023_schedule_record.db')

# parse the database and print the names of the tables

cur = con.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
for row in cur.fetchall():
    print(row[0])
    
    
# con.commit()


: 

In [13]:
# Dictionary to hold dataframes for each year
team_data = {}

for year in range(2001, 2009):  # 2009 because range is exclusive on the upper bound
    team_data[year] = pyb.team_ids(year)
# Function to compare two dataframes and detect changes
def compare_teams(df1, df2, year1, year2):
    merged = pd.merge(df1, df2, on='teamIDbr', how='outer', indicator=True)
    changes = merged[merged['_merge'] != 'both']
    print(f"Changes from {year1} to {year2}:")
    print(changes[['teamIDbr', '_merge']])
    print("\n")

# Loop through the years and compare consecutive years
for year in range(2001, 2008):
    compare_teams(team_data[year], team_data[year + 1], year, year + 1)


KeyError: 'teamIDbr'

In [11]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('Betting/data/databases/2023_schedule_record.db')

# Get the list of table names (i.e., team names)
query = "SELECT name FROM sqlite_master WHERE type='table';"
team_tables = pd.read_sql_query(query, conn)

# Initialize an empty DataFrame to store all game records
data = pd.DataFrame()

# Loop through each table name and fetch its contents
for table_name in team_tables['name']:
    team_data = pd.read_sql_query(f"SELECT * FROM `{table_name}`", conn)
    data = pd.concat([data, team_data], ignore_index=True)

# Close the database connection
#conn.close()

# Display the combined DataFrame
print(data.sample(15))

#data.to_csv('Betting/data/databases/2023_season.csv', index=False)
    

                    Date   Tm Home_Away  Opp   W/L    R    RA   Inn    W-L  \
4974    Wednesday, Aug 9  SDP         @  SEA     L  1.0   6.0   9.0  55-60   
3515  Tuesday, Aug 8 (2)  PHI      Home  WSN     L  4.0   5.0   9.0  62-52   
3246    Wednesday, Apr 5  MIA      Home  MIN     W  5.0   2.0   9.0    3-4   
4634    Saturday, Jul 22  ARI         @  CIN     L  2.0   4.0   9.0  54-45   
3913    Thursday, Apr 27  STL         @  SFG     W  6.0   0.0   9.0  10-16   
65     Wednesday, Jun 14  WSN         @  HOU  L-wo  4.0   5.0   9.0  26-40   
4229   Wednesday, Apr 19  CIN      Home  TBR     L  0.0   8.0   9.0   7-11   
4683    Thursday, Sep 14  ARI         @  NYM     L  1.0  11.0   9.0  76-72   
605      Tuesday, Aug 15  NYY         @  ATL     L  0.0   5.0   9.0  60-60   
3001    Wednesday, Jul 5  NYM         @  ARI     W  2.0   1.0   9.0  40-46   
2707      Friday, Aug 11  OAK         @  WSN     L  2.0   8.0   9.0  33-83   
3820     Tuesday, Jul 18  MIL         @  PHI     L  3.0   4.0   

In [25]:
import sqlite3
import pandas as pd

  # Assuming you have a list of team names

all_data = []  # List to hold data from each query

# Loop through each team and year, querying the database and collecting the data
for team in team_names:
    for year in range(2001, 2025):  # Loop through years 2000 to 2024
        db_path = f'Betting/data/databases/{year}_schedule_record.db'
        conn = sqlite3.connect(db_path)
        query = f'SELECT * FROM {team}'
        team_data = pd.read_sql_query(query, conn)
        conn.close()
        all_data.append(team_data)

# Concatenate all DataFrames in the list into a single DataFrame
data = pd.concat(all_data, ignore_index=True)

# Drop the columns that you don't need
columns_to_drop = ['index', 'Rk', 'G', 'Date', 'Time', 'Day', 'Venue', 'Attendance', 'Length', 
                   'Inning1Top', 'Inning1Bot', 'Inning2Top', 'Inning2Bot', 'Inning3Top', 'Inning3Bot', 
                   'Inning4Top', 'Inning4Bot', 'Inning5Top', 'Inning5Bot', 'Inning6Top', 'Inning6Bot', 
                   'Inning7Top', 'Inning7Bot', 'Inning8Top', 'Inning8Bot', 'Inning9Top', 'Inning9Bot', 
                   'Inning10Top', 'Inning10Bot', 'Inning11Top', 'Inning11Bot', 'Inning12Top', 'Inning12Bot']
data.drop(columns=columns_to_drop, inplace=True)

# Now data is ready for analysis or further processing


DatabaseError: Execution failed on sql 'SELECT * FROM BOS': no such table: BOS

In [12]:
# Parse dates
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Handle categorical variables
le_team = LabelEncoder()
data['Tm'] = le_team.fit_transform(data['Tm'])
data['Opp'] = le_team.transform(data['Opp'])  # Assuming same encoder as 'Tm'
data['Home_Away'] = data['Home_Away'].map({'Home': 1, 'Away': 0})  # Binary encoding

# Fill missing values
data['Home_Away'].fillna(0,inplace=True)
data['Attendance'].fillna(0,inplace=True)
data = data.drop(['Date','Orig. Scheduled'], axis=1)


C:\Users\rhoffpauir\AppData\Local\Temp\ipykernel_19104\1119635649.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
C:\Users\rhoffpauir\AppData\Local\Temp\ipykernel_19104\1119635649.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Home_Away'].fillna(0,inplace=True)
C:\Users\rhoffpauir\AppData\Local\Temp\ipykernel_1910

In [13]:
# Convert 'W/L' into a binary 'Is_Win' column
data['Is_Win'] = data['W/L'].apply(lambda x: 1 if 'W' in x else 0)

# Optionally, check the balance of win/loss
print(data['Is_Win'].value_counts())


Is_Win
0    2622
1    2562
Name: count, dtype: int64


In [14]:
data1 = pd.get_dummies(data)



In [15]:
from sklearn.model_selection import train_test_split

X = data.drop('Is_Win', axis=1)  # Features
y = data['Is_Win']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:


# Select features and target
features = ['Tm', 'Home_Away', 'Opp', 'cLI', 'Streak','D/N','Attendance']
target = 'R'  # Runs scored by the home team

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

# Initialize and train a RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicting scores
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = mse ** 0.5

print(f"Model RMSE: {rmse}")

ValueError: could not convert string to float: 'N'

In [33]:
rfc = RandomForestClassifier()

In [34]:
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [37]:
predictions = rfc.predict(X_test)

In [38]:
# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = mse ** 0.5

print(f"Model RMSE: {rmse}")

Model RMSE: 3.1888486992958707
